In [1]:
import os
from owlready2 import *
import pandas as pd
from rdflib import Graph

def convert_to_owl(input_path, output_owl_path): 
    g = Graph()
    input_format = 'ttl' if input_path.endswith('.ttl') else 'xml'
    g.parse(input_path, format=input_format)
    # Serialize the graph to RDF/XML format (commonly used for .owl files)
    g.serialize(destination=output_owl_path, format='pretty-xml')

def count_ontology_elements(ontology_path):
    try:
         
        onto = get_ontology(ontology_path).load()
        
        # Count the elements
        num_classes = len(list(onto.classes()))
        num_individuals = len(list(onto.individuals()))
        num_object_properties = len(list(onto.object_properties()))
        num_data_properties = len(list(onto.data_properties()))
        
        counts = {
            "filename": os.path.basename(ontology_path),
            "classes": num_classes,
            "individuals": num_individuals,
            "object_properties": num_object_properties,
            "data_properties": num_data_properties
        }
        
        return counts
    
    except Exception as e:
        print(f"Error loading {ontology_path}: {e}")
        return None

def count_elements_in_directory(directory_path): 
    ontology_counts = {}
    
    # Supported ontology file extensions
    supported_extensions = (".owl", ".ttl", ".rdf")
    
    # Create a directory for converted files if it doesn't exist
    converted_dir = os.path.join(directory_path, 'converted_files_bkp')
    os.makedirs(converted_dir, exist_ok=True)
     
    for filename in os.listdir(directory_path):
        if filename.endswith(supported_extensions):
            ontology_path = os.path.join(directory_path, filename)
            
            # If the file is .ttl or .rdf, convert it to .owl and move it to 'converted_files_bkp' folder
            if filename.endswith(('.ttl', '.rdf')): 
                converted_path = os.path.join(directory_path, os.path.splitext(filename)[0] + '.owl')
                convert_to_owl(ontology_path, converted_path)
                
                # move ttl / rdf to converted_files_bkp
                new_original_path = os.path.join(converted_dir, filename)
                os.rename(ontology_path, new_original_path)
                
                 
                ontology_path = converted_path
            
            # Count classes/properties in the .owl file
            counts = count_ontology_elements(ontology_path)
            if counts:
                ontology_counts[filename] = counts
    
    return ontology_counts 

In [12]:
directory_path = "../Data/assertion_evidence/" 
ontology_counts_assertion_evidence = count_elements_in_directory(directory_path)

df_assertion_evidence = pd.DataFrame.from_dict(ontology_counts_assertion_evidence,orient='index')
df_assertion_evidence_reset = df_assertion_evidence.reset_index(drop=True)
df_assertion_evidence_reset

,filename,classes,individuals,object_properties,data_properties
0,argo.owl,13,8,20,0
1,rdo_1.0.owl,15,0,36,1
2,nanopublication template ontology.owl,21,3,9,6
3,sio.owl,1584,0,211,1
4,amo.owl,8,0,21,0
5,rvo.owl,12,5,17,4
6,obi.owl,4924,304,85,5
7,evi.owl,30,3,39,21
8,hohv1.3.owl,468,18,194,4
9,eco.owl,3337,18,47,3


In [11]:
print(df_assertion_evidence_reset.to_string(index=False))

                             filename  classes  individuals  object_properties  data_properties
                             argo.owl       13            8                 20                0
                          rdo_1.0.owl       15            0                 36                1
nanopublication template ontology.owl       21            3                  9                6
                              sio.owl     1584            0                211                1
                              amo.owl        8            0                 21                0
                              rvo.owl       12            5                 17                4
                              obi.owl     4924          304                 85                5
                              evi.owl       30            3                 39               21
                          hohv1.3.owl      468           18                194                4
                              eco.owl   

In [15]:
directory_path = "../Data/provenance"
ontology_counts_provenance = count_elements_in_directory(directory_path)

df_provenance = pd.DataFrame.from_dict(ontology_counts_provenance,orient='index')
df_reset_provenance = df_provenance.reset_index(drop=True)
df_reset_provenance

Error loading ../Data/provenance/pav-ontology.owl: cannot access local variable 'entity' where it is not associated with a value


,filename,classes,individuals,object_properties,data_properties
0,gdprov_v0.7.owl,108,4,43,0
1,p-plan.owl,10,1,9,0
2,opmw3.1.owl,12,0,10,16
3,prov-o.owl,51,6,60,9


In [16]:
!pwd

/Users/tekrajchhetri/Documents/paper-writing-reading/assertion-evidence-paper/assertion-evidence-paper/Code


In [18]:
! ls $directory_path

converted_files_bkp opmw3.1.owl         pav-ontology.owl
gdprov_v0.7.owl     p-plan.owl          prov-o.owl
